In [1]:
import pandas as pd
import numpy as np
from sklearn.utils import check_random_state
from matplotlib import pyplot as plt
import seaborn as sns
sns.set_style("darkgrid")


In [2]:
n_grid = np.linspace(50,2000,16)
np.random.seed(0)
seeds = np.random.permutation(5000)[:1100]

# Functions

### Covariates

In [3]:
from scipy.stats import norm

def sample_norm_covariates(n_datapoints, random_seed):
    covariate = norm.rvs(1,0.3, size=n_datapoints, random_state = random_seed)
    covariate[covariate <0] = 0
    return covariate
    
def sample_uniform_covariates(n_datapoints,rng):
    covariate = rng.uniform(low=0, high=1.0, size=n_datapoints).round(decimals=6)
    return covariate

def sample_categorical_covariates(n_datapoints, rng, cat_number):
    covariate = rng.choice(cat_number, size=n_datapoints)
    return covariate

def sample_dataframe(n_datapoints, n_norm, n_unif, n_cat, random_seed):
    
    rng = np.random.RandomState(random_seed)
    
    norm_cov = np.transpose([sample_norm_covariates(n_datapoints,rng) for i in range(n_norm)])
    unif_cov = np.transpose([sample_uniform_covariates(n_datapoints,rng) for i in range(n_unif)])
    cat_cov = np.transpose([sample_categorical_covariates(n_datapoints,rng, 3) for i in range(n_cat)])
    
    df = pd.DataFrame(norm_cov)
    df = pd.concat([df,pd.DataFrame(unif_cov)],axis =1)
    df = pd.concat([df,pd.DataFrame(cat_cov)],axis =1)
    df = df.sample(frac = 1, axis = 1, random_state = random_seed)
    df.columns = [f'X{i}' for i in range(n_norm+n_unif+n_cat)]
    
    return df

### Hazards 

In [4]:
def weibull_hazard(t, k=1., s=1., t_shift=100, base_rate=1e2):
    # t_shift is a trick to avoid avoid negative powers at t=0 when k < 1.
    t = t + t_shift
    return base_rate * (k / s) * (t / s) ** (k - 1.)

def type1_hazards(df, t):
    baseline = weibull_hazard(t, k=0.003)
    #s = ((df["X0"]+df["X1"])/2 * (1 - (df["X2"]+df["X3"])/2 )).to_numpy()
    s = ((df["X0"]+df["X1"])/2 * (df["X2"]+df["X3"])/2 ).to_numpy()
    return s.reshape(-1, 1) * baseline.reshape(1, -1)

def type2_hazards(df, t):
    # Weibull hazards with k = 1 is just a constant over time:
    baseline = weibull_hazard(t, k=3, s=8e3)
    s = (
        ( (df["X4"]+df["X5"])/2 *  df["X6"]  + .001) * df["X7"]
    ).to_numpy()
    return s.reshape(-1, 1) * baseline.reshape(1, -1)

def type3_hazards(df, t):
    return np.vstack([
        0.5 * weibull_hazard(t, k=6 * x, s=4e3) * y
        for x, y in zip((df["X8"]+ df["X9"])/2, (df["X10"]+df["X11"])/2)
    ])

### Event Times

In [5]:
from scipy.stats import bernoulli

def sample_events_by_type(hazards,total_days, random_state=None):
    rng = check_random_state(random_state)
    outcomes = bernoulli.rvs(hazards, random_state=rng)
    any_event_mask = np.any(outcomes, axis=1)
    duration = np.full(outcomes.shape[0], fill_value=total_days)
    occurrence_rows, occurrence_cols = np.where(outcomes)
    # Some individuals might have more than one event occurrence,
    # we only keep the first one.
    # ex: trials = [[0, 0, 1, 0, 1]] -> duration = 2
    _, first_occurrence_idxs = np.unique(occurrence_rows, return_index=True)
    duration[any_event_mask] = occurrence_cols[first_occurrence_idxs]
    jitter = rng.rand(duration.shape[0])
    return pd.DataFrame(dict(event=any_event_mask, duration=duration + jitter))


def first_event(event_frames, event_ids, random_seed=None):
    rng = check_random_state(random_seed)
    event = np.zeros(event_frames[0].shape[0], dtype=np.int32)
    max_duration = np.max([ef["duration"].max() for ef in event_frames])
    duration = np.full_like(event_frames[0]["duration"], fill_value=max_duration)
    
    out = pd.DataFrame(
        {
            "event": event,
            "duration": duration,
        }
    )
    for event_id, ef in zip(event_ids, event_frames):
        mask = ef["event"] & (ef["duration"] < out["duration"])
        out.loc[mask, "event"] = event_id
        out.loc[mask, "duration"] = ef.loc[mask, "duration"]
    return out



### Censoring

In [6]:
def uniform_censoring(occurrences, censoring_weight=0.5, frac = 1, offset=0, random_state=None):
    n_datapoints = occurrences.shape[0]
    rng = check_random_state(random_state)
    max_duration = occurrences["duration"].max()
    censoring_durations = rng.randint(
        low=offset, high=max_duration/frac, size=n_datapoints
    )
    # reduce censoring randomly by setting durations back to the max,
    # effectively ensuring that a fraction of the datapoints will not
    # be censured.
    disabled_censoring_mask = rng.rand(n_datapoints) > censoring_weight
    censoring_durations[disabled_censoring_mask] = max_duration
    
    out = occurrences.copy()
    censor_mask = occurrences["duration"] > censoring_durations
    out.loc[censor_mask, "event"] = 0
    out.loc[censor_mask, "duration"] = censoring_durations[censor_mask]
    return out

### Generate Data

In [7]:
def sample_competing_events(data,total_days, uniform_censoring_weight=1.0, frac=1, max_observation_duration=2000, random_seed=None):
    
    rng = check_random_state(random_seed)
    t = np.linspace(0, total_days, total_days)
    
    hazard_funcs = [type1_hazards, type2_hazards, type3_hazards]
    event_ids = np.arange(len(hazard_funcs)) + 1
    all_hazards = np.asarray([hazard_func(data, t) for hazard_func in hazard_funcs])
    
    occurrences_by_type = [sample_events_by_type(all_hazards[i],total_days, random_state=rng) for i in range(all_hazards.shape[0])]
    occurrences = first_event(occurrences_by_type, event_ids)
    
    censored_occurrences = uniform_censoring(occurrences, censoring_weight=uniform_censoring_weight, frac = frac, random_state=rng)
    
    if max_observation_duration is not None:
        # censor all events after max_observation_duration
        max_duration_mask = censored_occurrences["duration"] > max_observation_duration
        censored_occurrences.loc[max_duration_mask, "duration"] = max_observation_duration
        censored_occurrences.loc[max_duration_mask, "event"] = 0
    return (
        censored_occurrences,
        occurrences,
        all_hazards  # shape = (n_event_types, n_observations, n_timesteps)
    )

In [8]:
def sample_data(n_datapoints,total_days,frac, seed):
    data = sample_dataframe(n_datapoints, 4, 4, 4, seed) 
    (events, events_uncensored, all_hazards) = sample_competing_events(data, total_days, frac=frac, random_seed= seed)
    data['event'] = pd.Series(events['event'])
    data['event'] = data['event'].replace(2,1)
    data['event'] = data['event'].replace(3,1)
    data['duration'] = pd.Series(events['duration'])
    
    return data

# Number of Samples

In [9]:
for N in n_grid:
    print(f'Number of samples: {N}')
    sd = 0 
    for seed_ in seeds:

        data = sample_data(int(N),1000,0.45,seed_)
        perc = np.sum(data['event']==0)*100/len(data['event'])
        if (48. <= perc <= 52.):
            sd = sd+1
            data.to_pickle(f"numsampledata_{int(N)}_{sd}.pkl")
            

    print(f'Number of simulation: {sd}')
    print('')



Number of samples: 50.0
Number of simulation: 240

Number of samples: 180.0
Number of simulation: 131

Number of samples: 310.0
Number of simulation: 140

Number of samples: 440.0
Number of simulation: 126

Number of samples: 570.0
Number of simulation: 133

Number of samples: 700.0
Number of simulation: 137

Number of samples: 830.0
Number of simulation: 108

Number of samples: 960.0
Number of simulation: 120

Number of samples: 1090.0
Number of simulation: 125

Number of samples: 1220.0
Number of simulation: 107

Number of samples: 1350.0
Number of simulation: 126

Number of samples: 1480.0
Number of simulation: 121

Number of samples: 1610.0
Number of simulation: 128

Number of samples: 1740.0
Number of simulation: 116

Number of samples: 1870.0
Number of simulation: 115

Number of samples: 2000.0
Number of simulation: 123



In [10]:
import os

for N in n_grid:
    N = int(N)
    for i in range(101,240):
        if os.path.exists(f'numsampledata_{N}_{i}.pkl'):
            os.remove(f'numsampledata_{N}_{i}.pkl') 

# Type of Failure

### N = 50

In [12]:
total_days = 1000
frac = 0.45
seed = seeds[0]
data = sample_dataframe(50, 4, 4, 4, seed) 
(events, events_uncensored, all_hazards) = sample_competing_events(data, total_days, frac=frac, random_seed= seed)
data['event'] = pd.Series(events['event'])
#data['event'] = data['event'].replace(2,1)
#data['event'] = data['event'].replace(3,1)
#data['duration'] = pd.Series(events['duration'])
data.head()

,X0,X1,X2,X3,X4,X5,X6,X7,X8,X9,X10,X11,event
0,1,0.238420,0.456759,0.350232,0.633816,0,0.097091,2,0.953831,0,0.076760,1.148564,0
1,2,1.053234,0.767023,0.337347,0.264125,1,0.717579,0,1.235120,1,0.475158,1.239502,1
2,0,0.904500,0.851456,0.106689,0.040769,1,0.744982,1,0.573726,2,0.045364,0.693150,0
3,2,1.040238,1.414311,0.728582,0.206343,0,0.503682,1,0.982463,2,0.762931,1.587294,1
4,1,1.233334,1.070550,0.580723,0.840705,2,0.797719,2,0.964565,1,0.420637,1.394509,2


In [13]:
perc = np.sum(data['event']==0)*100/len(data['event'])
perc

48.0

In [18]:
num_event = np.sum(data['event']!=0)

In [19]:
np.sum(data['event']==1)*100/num_event

73.07692307692308

In [20]:
np.sum(data['event']==2)*100/num_event

7.6923076923076925

In [21]:
np.sum(data['event']==3)*100/num_event

19.23076923076923

### N=2000

In [44]:
total_days = 1000
frac = 0.45
seed = seeds[11]
data = sample_dataframe(2000, 4, 4, 4, seed) 
(events, events_uncensored, all_hazards) = sample_competing_events(data, total_days, frac=frac, random_seed= seed)
data['event'] = pd.Series(events['event'])
#data['event'] = data['event'].replace(2,1)
#data['event'] = data['event'].replace(3,1)
#data['duration'] = pd.Series(events['duration'])
data.head()

,X0,X1,X2,X3,X4,X5,X6,X7,X8,X9,X10,X11,event
0,1.064788,0.924189,2,1.208070,0,0.570211,0.525933,1.111118,0.431915,0.037417,0,1,3
1,0.640560,0.489096,1,0.282401,0,0.611204,0.332478,0.564717,1.037840,0.508528,2,2,0
2,0.709673,0.285023,2,0.766734,1,0.557823,0.255944,0.748691,1.104296,0.109941,0,1,1
3,0.712869,0.102668,2,1.311409,1,0.992587,0.440197,1.480252,1.322194,0.620255,0,0,0
4,0.918879,0.963147,0,1.383394,1,0.857596,0.850684,1.152569,0.494228,0.396105,2,0,1


In [45]:
perc = np.sum(data['event']==0)*100/len(data['event'])
perc

51.65

In [46]:
num_event = np.sum(data['event']!=0)

In [47]:
np.sum(data['event']==1)*100/num_event

70.63081695966908

In [48]:
np.sum(data['event']==2)*100/num_event

8.479834539813858

In [49]:
np.sum(data['event']==3)*100/num_event

20.889348500517062